### Recommendations Rank & Knowledge Based: 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [2]:
movies.shape

(31245, 35)

In [3]:
reviews.shape

(712337, 23)

In [4]:
movies.head(2)

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


In [5]:
reviews.head(2)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best✅ **(2)**
* With ties, movies that have more ratings are better✅ **(3)**
* A movie must have a minimum of 5 ratings to be considered among the best movies ✅ **(1)**
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating ✅

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [6]:
gr_than5_inds=reviews.movie_id.value_counts().index[reviews.movie_id.value_counts()>=5]
gr_than5_rev= reviews[reviews['movie_id'].isin(gr_than5_inds)]
gr_than5_rev.movie_id.value_counts().min()

5

In [7]:
rating_based = gr_than5_rev.groupby('movie_id').agg({'rating':'mean'}).sort_values(by='rating',ascending=False).reset_index()
rating_based

,movie_id,rating
0,5131914,10.000000
1,2219210,10.000000
2,4921860,10.000000
3,2737018,10.000000
4,5262972,10.000000
...,...,...
9969,60666,1.857143
9970,3108604,1.800000
9971,3187378,1.666667
9972,4458206,1.000000


In [8]:
# best= rating_based[['movie_id','rating']][:5]
# best

In [9]:
# vc=gr_than5_rev['movie_id'][gr_than5_rev['movie_id'].isin(best['movie_id'])].value_counts().reset_index()
# vc.columns=['movie_id','count']
# vc

vc=gr_than5_rev['movie_id'].value_counts().reset_index()
vc.columns=['movie_id','count']
vc

,movie_id,count
0,1454468,3029
1,993846,2673
2,770828,2617
3,816692,2572
4,816711,2385
...,...,...
9969,46889,5
9970,15175,5
9971,489037,5
9972,59740,5


In [10]:
gr_than5_rev['date']= pd.to_datetime(gr_than5_rev['date'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
date_based= gr_than5_rev.groupby('movie_id').date.max().reset_index()
date_based

,movie_id,date
0,417,2018-06-26 21:08:46
1,6864,2017-10-22 00:06:30
2,10323,2017-09-29 06:22:04
3,12349,2018-05-14 03:18:44
4,12364,2016-09-30 18:20:04
...,...,...
9969,7830574,2018-06-24 22:54:17
9970,7853242,2018-06-23 06:50:31
9971,7923374,2018-02-24 03:21:51
9972,7983844,2018-04-11 03:10:21


In [12]:
newbest= rating_based.merge(vc, on='movie_id').merge(date_based, on='movie_id')
print(newbest.shape)
newbest

(9974, 4)


,movie_id,rating,count,date
0,5131914,10.000000,5,2016-04-04 05:52:22
1,2219210,10.000000,6,2013-07-22 23:30:52
2,4921860,10.000000,48,2016-08-14 17:16:50
3,2737018,10.000000,10,2015-05-10 22:56:01
4,5262972,10.000000,28,2016-01-08 00:44:43
...,...,...,...,...
9969,60666,1.857143,7,2018-05-06 19:18:10
9970,3108604,1.800000,5,2017-10-31 03:02:32
9971,3187378,1.666667,6,2017-07-11 06:13:16
9972,4458206,1.000000,30,2018-01-20 04:44:12


In [13]:
newbest= newbest.sort_values(by=['rating','count','date'], ascending=False)

In [14]:
last_df= newbest.merge(movies,on='movie_id')
last_df

,movie_id,rating,count,date_x,movie,genre,date_y,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,4921860,10.000000,48,2016-08-14 17:16:50,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
1,5262972,10.000000,28,2016-01-08 00:44:43,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
2,5688932,10.000000,14,2018-06-17 01:44:48,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0
3,2737018,10.000000,10,2015-05-10 22:56:01,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,2560840,10.000000,6,2016-01-23 00:30:44,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9969,60666,1.857143,7,2018-05-06 19:18:10,Manos: The Hands of Fate (1966),Horror,1966,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9970,3108604,1.800000,5,2017-10-31 03:02:32,American Poltergeist (2015),Thriller,2015,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9971,3187378,1.666667,6,2017-07-11 06:13:16,The Asian Connection (2016),Action,2016,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9972,4458206,1.000000,30,2018-01-20 04:44:12,Kod Adi K.O.Z. (2015),Crime|Mystery,2015,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
list(last_df['movie'][:5])

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)"]

In [16]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    gr_than5_inds=reviews.movie_id.value_counts().index[reviews.movie_id.value_counts()>=5]
    gr_than5_rev= reviews[reviews['movie_id'].isin(gr_than5_inds)]
    # gr_than5_rev['date']= pd.to_datetime(gr_than5_rev['date'])


    rating_based = gr_than5_rev.groupby('movie_id').agg({'rating':'mean'}).sort_values(by='rating',ascending=False).reset_index()

    
    counts_df =gr_than5_rev.movie_id.value_counts().reset_index()
    counts_df.columns=['movie_id','count']
    date_based= gr_than5_rev.groupby('movie_id').date.max().reset_index()
    
    df= rating_based.merge(counts_df, on='movie_id').merge(date_based, on='movie_id').sort_values(by=['rating','count','date'], ascending=False)
    
    
    final_df= df.merge(movies,on='movie_id')

    top_movies= list(final_df['movie'][:n_top])


    # tie= rating_based[:n_top]['rating'].nunique()<n_top
    
    # if tie:
    #   best_n= gr_than5_rev['movie_id'][gr_than5_rev['movie_id'].isin(best_n)].value_counts().index




    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [17]:
popular_recommendations("53968",5)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)"]

In [18]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations("1",20)

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations("53968",5)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations("70000",100)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations("43",35)



In [19]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy'])
```

In [20]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [29]:
movies['genre']= movies['genre'].astype(str)

In [30]:
movies['genre'] = movies['genre'].apply(lambda x: x.split('|'))

In [32]:
# movies.dtypes

In [63]:
def popular_recs_filtered(user_id, n_top, years=0, genres=0):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    gr_than5_inds=reviews.movie_id.value_counts().index[reviews.movie_id.value_counts()>=5]
    gr_than5_rev= reviews[reviews['movie_id'].isin(gr_than5_inds)]

    rating_based = gr_than5_rev.groupby('movie_id').agg({'rating':'mean'}).sort_values(by='rating',ascending=False).reset_index()

    
    counts_df =gr_than5_rev.movie_id.value_counts().reset_index()
    counts_df.columns=['movie_id','count']
    date_based= gr_than5_rev.groupby('movie_id').date.max().reset_index()
    
    df= rating_based.merge(counts_df, on='movie_id').merge(date_based, on='movie_id').sort_values(by=['rating','count','date'], ascending=False)
    
    
    if genres==0 and years==0:
      new_movies_df= movies

    elif genres==0 and years!=0:
      new_movies_df= movies[movies['date'].isin(years)]

    elif genres!=0:
      mask= movies.genre.apply(lambda x: any(g in x for g in genres))
      if years==0:
        new_movies_df= movies[mask]
      elif years!=0:
        new_movies_df= movies[mask & (movies['date'].isin(years))]
    
    
    final_df= df.merge(new_movies_df,on='movie_id')

    
      # final_df= sub_final_df

    # elif genres!=0 and years!=0:
    #   final_df= sub_final_df[(sub_final_df['date_y'].isin(years)) | (sub_final_df['genre'].isin(genres))]

    # elif genres!=0 and years==0:
    #   final_df= sub_final_df[sub_final_df['genre'].isin(genres)]

    # elif genres==0 and years!=0:
    #   final_df= sub_final_df[sub_final_df['date_y'].isin(years)]


    top_movies= list(final_df['movie'][:n_top])




    return top_movies # a list of the n_top movies as recommended

In [60]:
# def popular_recs_filtered(user_id, n_top, years=0, genres=0):
#   recommended = popular_recommendations(user_id, n_top)
#   if years==0 and genres==0:
#     top_movies= recommended
#   elif years==0 and genres!=0:
#     top_movies= list(last_df['movie'][(last_df['movie'].isin(recommended)) & (last_df['genre'].isin(genres))])
#   elif years!=0 and genres==0:
#     top_movies= list(last_df['movie'][(last_df['movie'].isin(recommended)) & (last_df['date_y'].isin(years))])
#   elif years!=0 and genres!=0:
#     top_movies= list(last_df['movie'][(last_df['movie'].isin(recommended)) & ((last_df['genre'].isin(genres)) | (last_df['date_y'].isin(years)))])
  
#   return top_movies



In [61]:
# m=movies[movies['movie']=='MSG 2 the Messenger (2015)']

In [64]:
popular_recs_filtered('1', 20, years=[2015, 2016, 2017, 2018], genres=['History', 'Comedy'])

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Make Like a Dog (2015)',
 'Be Somebody (2016)',
 'Poshter Girl (2016)',
 'Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'Bajrangi Bhaijaan (2015)',
 'The Farthest (2017)',
 'Ricky Gervais: Humanity (2018)',
 'Coco (2017)',
 'The Age of Spin: Dave Chappelle Live at the Hollywood Palladium (2017)',
 'Sado (2015)',
 'Adult Life Skills (2016)',
 'Hatred (2016)',
 'Inside Out (2015)',
 'La vida inmoral de la pareja ideal (2016)',
 'World of Tomorrow (2015)',
 'Toc Toc (2017)']

In [37]:
print(t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy']))

['MSG 2 the Messenger (2015)', 'Avengers: Age of Ultron Parody (2015)', 'Sorry to Bother You (2018)', 'Make Like a Dog (2015)', 'Be Somebody (2016)', 'Poshter Girl (2016)', 'Ayla: The Daughter of War (2017)', 'I Believe in Miracles (2015)', 'Bajrangi Bhaijaan (2015)', 'The Farthest (2017)', 'Ricky Gervais: Humanity (2018)', 'Coco (2017)', 'The Age of Spin: Dave Chappelle Live at the Hollywood Palladium (2017)', 'Sado (2015)', 'Adult Life Skills (2016)', 'Hatred (2016)', 'Inside Out (2015)', 'La vida inmoral de la pareja ideal (2016)', 'World of Tomorrow (2015)', 'Toc Toc (2017)']


In [58]:
movies[movies['movie']=='Ayla: The Daughter of War (2017)']

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
30587,6316138,Ayla: The Daughter of War (2017),"[Drama, History, War]",2017,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [66]:
recs_20_with_year_genre= popular_recs_filtered('1', 20, years=[2015, 2016, 2017, 2018], genres=['History', 'Comedy'])

# check 
assert t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy']) == recs_20_with_year_genre,  "Check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!
